<a href="https://colab.research.google.com/github/jhuarancca/Applied-AI-Building-NLP-Apps-with-Hugging-Face-Transformers/blob/main/Code_04_XX_Building_a_custom_Model_with_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()


In [3]:

!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


## 04.02. Loading a Hugging Face Dataset

In [5]:
from datasets import load_dataset

#Use pretrained model checkpoint from Huggingface
model_name = "distilbert-base-uncased"
#Use pre-labeled dataset from huggingface
dataset_name= "poem_sentiment"

poem_sentiments = load_dataset(dataset_name)

#Apache Arrow format
print(poem_sentiments)
print(poem_sentiments["test"][20:25])

print("\nSentiment Labels used",
      poem_sentiments["train"].features.get("label").names)

TypeError: http_get() got an unexpected keyword argument 'displayed_filename'

## 04.03. Encoding and pre-processing the dataset

In [6]:
#Encoding text

from transformers import DistilBertTokenizer

db_tokenizer = DistilBertTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return db_tokenizer(batch["verse_text"],
                        padding=True,
                        truncation=True)

enc_poem_sentiment = poem_sentiments.map(
                        tokenize,
                        batched=True,
                        batch_size=None)

print(enc_poem_sentiment["train"][0:5])


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

NameError: name 'poem_sentiments' is not defined

In [ ]:
#Explore input IDs and Attention Mask

print("Text :",
      enc_poem_sentiment["train"][1].get("verse_text"))
print("\nInput Map :",
      enc_poem_sentiment["train"][1].get("input_ids"))
print("\nAttention Mask :",
      enc_poem_sentiment["train"][1].get("attention_mask"))

print("\nTotal tokens: ",
      len(enc_poem_sentiment["train"][1].get("input_ids")))
print("Non Zero tokens: ",
      len(list(filter(
        lambda x :x > 0,
          enc_poem_sentiment["train"][1].get("input_ids")))))
print("Attention = 1: ",
      len(list(filter(
        lambda x :x > 0,
          enc_poem_sentiment["train"][1].get("attention_mask")))))

Text : it flows so long as falls the rain,

Input Map : [101, 2009, 6223, 2061, 2146, 2004, 4212, 1996, 4542, 1010, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Attention Mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Total tokens:  28
Non Zero tokens:  11
Attention = 1:  11


In [ ]:
#Separate training and validation sets
training_dataset = enc_poem_sentiment["train"]
validation_dataset=enc_poem_sentiment["validation"]

print("\nColumn Names : ",training_dataset.column_names)
print("\nFeatures : ",training_dataset.features)

labels = training_dataset.features.get("label")
num_labels=len(labels.names)



Column Names :  ['id', 'verse_text', 'label', 'input_ids', 'attention_mask']

Features :  {'id': Value(dtype='int32', id=None), 'verse_text': Value(dtype='string', id=None), 'label': ClassLabel(names=['negative', 'positive', 'no_impact', 'mixed'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


## 04.04. Creating the Model Architecture

In [ ]:
from transformers import TFAutoModelForSequenceClassification

#Load transformer checkpoint from huggingface
sentiment_model = (TFAutoModelForSequenceClassification
            .from_pretrained(model_name, num_labels=num_labels))

sentiment_model.get_config()


{'vocab_size': 30522,
 'max_position_embeddings': 512,
 'sinusoidal_pos_embds': False,
 'n_layers': 6,
 'n_heads': 12,
 'dim': 768,
 'hidden_dim': 3072,
 'dropout': 0.1,
 'attention_dropout': 0.1,
 'activation': 'gelu',
 'initializer_range': 0.02,
 'qa_dropout': 0.1,
 'seq_classif_dropout': 0.2,
 'return_dict': True,
 'output_hidden_states': False,
 'output_attentions': False,
 'torchscript': False,
 'torch_dtype': None,
 'use_bfloat16': False,
 'tf_legacy_loss': False,
 'pruned_heads': {},
 'tie_word_embeddings': True,
 'is_encoder_decoder': False,
 'is_decoder': False,
 'cross_attention_hidden_size': None,
 'add_cross_attention': False,
 'tie_encoder_decoder': False,
 'max_length': 20,
 'min_length': 0,
 'do_sample': False,
 'early_stopping': False,
 'num_beams': 1,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'typical_p': 1.0,
 'repetition_penalty': 1.0,
 'length_penalty': 1.0,
 'no_repeat_ngram_size': 0,
 'encoder_no_repeat_ng

In [ ]:
#Freeze the first layer if needed
sentiment_model.layers[0].trainable = True

#Add/remove layers if needed.
#sentiment_model.layers [append()/insert()/remove()]

print(sentiment_model.summary())



Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  3076      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,956,548
Trainable params: 66,956,548
Non-trainable params: 0
_________________________________________________________________
None


# 04.05. Training the Sentiment Model

In [ ]:
#Using features from a pretrained model

import tensorflow as tf

batch_size=64
tokenizer_columns = db_tokenizer.model_input_names

# Convert to TF tensors
train_dataset = training_dataset.to_tf_dataset(
    columns=tokenizer_columns, label_cols=["label"], shuffle=True,
    batch_size=batch_size)
val_dataset = validation_dataset.to_tf_dataset(
    columns=tokenizer_columns, label_cols=["label"], shuffle=False,
    batch_size=batch_size)

sentiment_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy())

sentiment_model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=5)

Epoch 1/5
14/14 [==============================] - 72s 4s/step - loss: 0.2459 - sparse_categorical_accuracy: 0.9204 - val_loss: 0.5771 - val_sparse_categorical_accuracy: 0.8571
Epoch 2/5
14/14 [==============================] - 59s 4s/step - loss: 0.1215 - sparse_categorical_accuracy: 0.9675 - val_loss: 0.4355 - val_sparse_categorical_accuracy: 0.8762
Epoch 3/5
14/14 [==============================] - 59s 4s/step - loss: 0.0808 - sparse_categorical_accuracy: 0.9753 - val_loss: 0.4960 - val_sparse_categorical_accuracy: 0.8857
Epoch 4/5
14/14 [==============================] - 59s 4s/step - loss: 0.0474 - sparse_categorical_accuracy: 0.9899 - val_loss: 0.5431 - val_sparse_categorical_accuracy: 0.8762
Epoch 5/5
14/14 [==============================] - 60s 4s/step - loss: 0.0249 - sparse_categorical_accuracy: 0.9955 - val_loss: 0.5467 - val_sparse_categorical_accuracy: 0.8857


## 04.06. Predicting Sentiment with the Custom Model

In [ ]:
from datasets import Dataset,DatasetDict

#Input data for interference to predict sentiment
# the "label" array is not needed for inference, but added to provide true labels for comparison
infer_data = {'id':[0,1],
             'verse_text':['and be glad in the summer morning when the kindred ride on their way',
                           'how hearts were answering to his own'],
             'label':[1,0]}

infer_dataset = Dataset.from_dict(infer_data)

ds_dict=DatasetDict()
ds_dict["infer"] = infer_dataset

print(ds_dict)

#Encode the dataset, similar to training
enc_dataset=ds_dict.map(tokenize, batched=True, batch_size=None)

#Convert to Tensors
infer_final_dataset = enc_dataset["infer"].to_tf_dataset(
    columns=tokenizer_columns,  shuffle=True,
    batch_size=batch_size)

print(infer_final_dataset)

#Predict with the model
predictions=sentiment_model.predict(infer_final_dataset)

DatasetDict({
    infer: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 2
    })
})


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 189.46ba/s]


<PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(None, 17), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 17), dtype=tf.int64, name=None)}>
1/1 [==============================] - 0s 138ms/step


In [ ]:
predictions.logits

array([[-2.3346035,  4.137719 , -1.7129751, -1.0897871],
       [-2.544504 ,  3.4136968, -0.5179995, -1.3775116]], dtype=float32)

In [ ]:
import numpy as np
pred_label_ids=np.argmax(predictions.logits, axis=1)

for i in range(len(pred_label_ids)):
    print("Poem =", infer_data["verse_text"][i],
          " Predicted=",labels.names[pred_label_ids[i]],
          " True-Label=",labels.names[infer_data["label"][i]])

Poem = and be glad in the summer morning when the kindred ride on their way  Predicted= positive  True-Label= positive
Poem = how hearts were answering to his own  Predicted= positive  True-Label= negative
